<a href="https://colab.research.google.com/github/goerlitz/nlp-classification/blob/main/notebooks/10kGNAD/colab/21_10kGNAD_simpletransformers_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying German News Articles with SimpleTransformers

## Objectives

1. Train a text classifier with transfer learning based on a pretrained German transformer model.
2. Keep the implementation simple (just a few lines of code) by using the SimpleTransformers library. It also has sensible default model settings.


## Approach

Use following pretrained models on the 10k German News Articles dataset to classify 9 news topics.

* `distilbert-base-german-cased`
* `deepset/gbert-base`
* `deepset/gelectra-large`

## Learnings

...

## Prerequisites

In [5]:
# model_type = "distilbert"
# model_name = "distilbert-base-german-cased"

# model_type = "bert"
# model_name = "deepset/gbert-base"

model_type = "electra"
model_name = "deepset/gelectra-large"

project_name = "10kgnad_default__" + model_name.replace("/", "_")

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jun 15 21:51:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# install transformers
!pip install -q -U tqdm==4.47.0 transformers simpletransformers >/dev/null

# check installed version
!pip freeze | grep transformers
!pip freeze | grep torch
# simpletransformers==0.61.6
# transformers==4.6.1
# torch==1.8.1+cu101

ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
simpletransformers==0.61.6
transformers==4.6.1
torch==1.8.1+cu101
torchsummary==1.5.1
torchtext==0.9.1
torchvision==0.9.1+cu101


In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

from simpletransformers.classification import ClassificationModel
from transformers import AutoTokenizer
from transformers import logging
import wandb

# hide progress bar when downloading tokenizers - a workaround!
logging.get_verbosity = lambda : logging.NOTSET

# disable transformer warnings like "Some weights of the model checkpoint"
logging.set_verbosity_error()

# disable logging of wandb
os.environ["WANDB_SILENT"] = "true"

## Download Data

Using the [10k German News Articles Dataset](https://tblock.github.io/10kGNAD/)

In [9]:
%env DIR=data

!mkdir -p $DIR
!wget -nc https://github.com/tblock/10kGNAD/blob/master/train.csv?raw=true -nv -O $DIR/train.csv
!wget -nc https://github.com/tblock/10kGNAD/blob/master/test.csv?raw=true -nv -O $DIR/test.csv
!ls -lAh $DIR | cut -d " " -f 5-

env: DIR=data
2021-06-15 21:52:27 URL:https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv [24405789/24405789] -> "data/train.csv" [1]
2021-06-15 21:52:29 URL:https://raw.githubusercontent.com/tblock/10kGNAD/master/test.csv [2755020/2755020] -> "data/test.csv" [1]

2.7M Jun 15 21:52 test.csv
 24M Jun 15 21:52 train.csv


## Import Data

In [10]:
data_dir = Path("data/")

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'

def read_csv_10kGNAD(filepath: Path, columns=["labels", "text"]) -> pd.DataFrame:
    """Load 10kGNAD csv file, handling its specific file format."""
    f = pd.read_csv(filepath, sep=";", quotechar="'", names=columns)
    return f

In [11]:
train_df = read_csv_10kGNAD(data_dir / 'train.csv')
print(train_df.shape[0], 'articles')
display(train_df.head())

9245 articles


,labels,text
0,Sport,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,Kultur,"Erfundene Bilder zu Filmen, die als verloren g..."
2,Web,Der frischgekürte CEO Sundar Pichai setzt auf ...
3,Wirtschaft,"Putin: ""Einigung, dass wir Menge auf Niveau vo..."
4,Inland,Estland sieht den künftigen österreichischen P...


In [12]:
test_df = read_csv_10kGNAD(data_dir / 'test.csv')
print(test_df.shape[0], 'articles')
display(test_df.head())

1028 articles


,labels,text
0,Wirtschaft,"Die Gewerkschaft GPA-djp lanciert den ""All-in-..."
1,Sport,Franzosen verteidigen 2:1-Führung – Kritische ...
2,Web,Neues Video von Designern macht im Netz die Ru...
3,Sport,23-jähriger Brasilianer muss vier Spiele pausi...
4,International,Aufständische verwendeten Chemikalie bei Gefec...


## Prepare Data for Model Training

There are a few requiremenf for feeding training data into SimpleTransformers:

* columns should be labeled `labels` and `text` (already done when reading the data)
* labels must be encoded as int values (starting at `0`!)

Additionally, we can handle imbalanced datasets by

* computing class weights for training

### Label Encoding

In [13]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(train: pd.DataFrame, test: pd.DataFrame):
    le = LabelEncoder()

    train_labels = le.fit_transform(train.labels)
    test_labels = le.transform(test.labels)

    return train.assign(labels=train_labels), test.assign(labels=test_labels)

train_df, test_df = encode_labels(train_df, test_df)
display(train_df.head())

,labels,text
0,5,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,3,"Erfundene Bilder zu Filmen, die als verloren g..."
2,6,Der frischgekürte CEO Sundar Pichai setzt auf ...
3,7,"Putin: ""Einigung, dass wir Menge auf Niveau vo..."
4,1,Estland sieht den künftigen österreichischen P...


### Computing Class Weights (not used yet)

In [14]:
from sklearn.utils.class_weight import compute_class_weight

def class_weights(labels: pd.Series) -> pd.DataFrame:
    """Compute class weights for imbalanced data."""
    uniq_labels = labels.unique()
    counts_s = labels.value_counts().reindex(uniq_labels)
    weights = compute_class_weight("balanced", uniq_labels, labels)
    return pd.DataFrame({"count": counts_s, "weight": weights}).sort_index()

weights_df = class_weights(train_df.labels)
display(weights_df)

,count,weight
0,601,1.709188
1,913,1.125106
2,1360,0.755310
3,485,2.117984
4,1510,0.680280
5,1081,0.950252
6,1509,0.680730
7,1270,0.808836
8,516,1.990741


## Model Setup

In [15]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')

def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average='macro')

def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average='macro')

In [16]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 200,    
    "evaluate_during_training_verbose": False,
    "evaluate_during_training_silent": True,
    "silent": True,
    "fp16": False,
    "wandb_project": project_name,
    }

In [17]:
def train():

    # need to create a tokenizer first and adjust train args with lower case setting
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    args = {**train_args, **{ "do_lower_case": tokenizer.do_lower_case }}

    # Create a ClassificationModel
    model = ClassificationModel(model_type,
                                model_name,
                                num_labels=train_df.labels.nunique(),
                                args=args)


    steps, details = model.train_model(train_df,
                                       eval_df=test_df,
                                       verbose=False,
                                       f1=f1_multiclass,
                                       acc=accuracy_score,
                                       precision=precision_multiclass,
                                       recall=recall_multiclass)
    
    print(details)

    wandb.join()

In [ ]:
while True:
    train()

<IPython.core.display.Javascript object>

{'global_step': [200, 400, 600, 800, 1000, 1156], 'mcc': [0.7545361920457722, 0.8375471307145497, 0.8322818327528577, 0.8516413564864762, 0.8484767507837376, 0.8663262942875827], 'train_loss': [0.8192497491836548, 0.9092986583709717, 0.09124189615249634, 0.03136887028813362, 0.060369957238435745, 0.7705817818641663], 'eval_loss': [0.7561756147779235, 0.5354693738036147, 0.573098985622673, 0.5377070864431502, 0.5133395715176251, 0.4432173087153324], 'f1': [0.754437049202068, 0.8581801800422251, 0.8487465175009733, 0.8658173333504677, 0.8604546974191454, 0.8774893410319188], 'acc': [0.7840466926070039, 0.8570038910505836, 0.853112840466926, 0.8696498054474708, 0.867704280155642, 0.8832684824902723], 'precision': [0.7777216002930692, 0.8684560165686497, 0.8522688041863253, 0.8820538659254636, 0.8772959428814561, 0.8835729074604011], 'recall': [0.7566428752280703, 0.858734730162415, 0.8501070852561736, 0.8593993765299731, 0.8510796955083573, 0.8738238994918135]}
{'global_step': [200, 400, 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'global_step': [200, 400, 600, 800, 1000, 1156], 'mcc': [0.7204902844066637, 0.8190498945895615, 0.8525430386609827, 0.8713928143610861, 0.8720873866374537, 0.8754467946571509], 'train_loss': [0.9709653258323669, 0.43063291907310486, 0.20738773047924042, 0.818006157875061, 0.043065089732408524, 0.01651296205818653], 'eval_loss': [0.9064495444528816, 0.6337928389625032, 0.4927543645161529, 0.43306925596828144, 0.4243406290188432, 0.413127032447924], 'f1': [0.6772647694343612, 0.817118344346651, 0.8593892421735334, 0.8824781712196804, 0.8834343923803277, 0.8876462330610112], 'acc': [0.75, 0.8414396887159533, 0.8706225680933852, 0.8871595330739299, 0.8881322957198443, 0.8910505836575876], 'precision': [0.6822177558029672, 0.842956452840205, 0.8607651120127588, 0.8840663686366712, 0.8822604248034857, 0.8867229247749606], 'recall': [0.7028311995336561, 0.8100441030761326, 0.8656010843857583, 0.8847866478456768, 0.8855426729409595, 0.8899664654555601]}
{'global_step': [200, 400, 600, 800, 1

---
## Evaluate Best Model

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score, precision=precision_multiclass, recall=recall_multiclass, wandb_log=False)
pd.Series(result)

In [ ]:
preds = pd.DataFrame(model_outputs, columns=mapping_s)
preds

In [ ]:
# preds.to_csv("data/predictions.csv", index=False)

In [ ]:
pred_s = pd.DataFrame(model_outputs).idxmax(axis=1)

In [ ]:
mapping_s.values

In [ ]:
import sklearn.metrics as skm
skm.confusion_matrix(test_df.labels, pred_s)

In [ ]:
print(skm.classification_report(test_df.labels, pred_s, target_names=mapping_s.values))